In [1]:
import pandas as pd
import random

from collections import OrderedDict
from itertools import islice


#### `(u^2 * w^2 * x * y) + (w * x * y * z^2) + (u^2 * w^2 * x * y^2) + (u  * x  * z) + y  = 40`

In [3]:
POPULATION_SIZE = 3000
MIN_VALUE = -300
MAX_VALUE = 300
NUM_ITEMS_FOR_SELECTION = 1000
NUM_ITEMS_FOR_MUTATION = 500
NUM_OF_SUITABLE_DESCEDANTS = 200

MUTATION_PROBABILITY_FOR_SUITABLE = 0.01
MUTATION_PROBABILITY_FOR_UNSUITABLE = 0.25

In [4]:
def generate_item(min_value, max_value):
    return [random.randint(min_value, max_value) for _ in ['u', 'w', 'x', 'y', 'z']]


In [5]:
def create_initial_population(population_size: int, min_value: int, max_value: int):
    population = []
    for index in range(population_size):
        item = generate_item(min_value, max_value)
        population.append(item)
    return population


In [6]:
population = create_initial_population(POPULATION_SIZE, MIN_VALUE, MAX_VALUE)

In [7]:
TARGET_VALUE = 40


def target_function(u, w, x, y, z):
    return abs(u**2*w**2*x*y + w*x*y*z**2 + u**2*w**2*x*y**2 + u*x*z + y - TARGET_VALUE)


In [8]:
def perform_tournament_selection(population, num_items_for_selection, num_items_for_mutation, target_function):
    sample =  random.sample(population, num_items_for_selection)
    
    target_item_dict = {
        target_function(*item): item
        for item in sample
    }
    ordered_target_item_dict = OrderedDict(sorted(target_item_dict.items()))
    
    return list(OrderedDict(islice(ordered_target_item_dict.items(), 0, num_items_for_mutation)).values())
    

In [149]:
selection = perform_tournament_selection(population, NUM_ITEMS_FOR_SELECTION, NUM_ITEMS_FOR_MUTATION, target_function)

In [9]:
def perform_homogeneous_crossing_for_two_elements(first_item, second_item):
    new_item = [
        random.choice([coordinate1, coordinate2]) for coordinate1, coordinate2 in zip(first_item, second_item)
    ]
    return new_item


In [10]:
def perform_homogeneous_crossing(selection, descendants_size):
    descendants = [
        perform_homogeneous_crossing_for_two_elements(random.choice(selection), random.choice(selection))
        for _ in range(descendants_size)
    ]
    return descendants


In [11]:
def should_perform_coordinate_mutation(mutation_probability):
    return random.random() < mutation_probability


In [12]:
def perform_mutation(
   selection, num_of_suitable_descendants, mutation_probability_for_suitable, mutation_probability_for_unsuitable
):
    
    for item in selection[:num_of_suitable_descendants]:
        for index, coordinate in enumerate(item):
            if should_perform_coordinate_mutation(mutation_probability_for_suitable):
                item[index] = random.randint(MIN_VALUE, MAX_VALUE)
    
    for item in selection[num_of_suitable_descendants:]:
        for index, coordinate in enumerate(item):
            if should_perform_coordinate_mutation(mutation_probability_for_unsuitable):
                item[index] = random.randint(MIN_VALUE, MAX_VALUE)

    return selection


In [168]:
selection = perform_mutation(
    selection,
    NUM_OF_SUITABLE_DESCEDANTS,
    MUTATION_PROBABILITY_FOR_SUITABLE,
    MUTATION_PROBABILITY_FOR_UNSUITABLE
)

In [13]:
def perform_descendants_substitution(selection, population_size, target_function):
    return perform_tournament_selection(selection, len(selection), population_size, target_function)


In [177]:
perform_descendants_substitution(selection, 2*POPULATION_SIZE, POPULATION_SIZE, target_function)[:5]

In [14]:
def run_genetic_algorithm(
    population_size, min_value, max_value, num_items_for_selection, num_items_for_mutation, target_function,
    num_of_suitable_descedants, mutation_probability_for_suitable, mutation_probability_for_unsuitable
):
    initial_population = create_initial_population(population_size, min_value, max_value)
    iteration = 0
    while True:
        selection = perform_tournament_selection(
            initial_population, num_items_for_selection, num_items_for_mutation, target_function
        )
        descendants = perform_homogeneous_crossing(selection, 2*population_size)
        descedants = perform_mutation(
            descendants,
            num_of_suitable_descedants,
            mutation_probability_for_suitable,
            mutation_probability_for_unsuitable
        )
        selection = perform_descendants_substitution(
            descendants, population_size, target_function
        )
        
        target_values = [target_function(*item) for item in selection]
        print(f"MIN of target values: {min(target_values)}; AVG of target values: {sum(target_values)/len(target_values)}")
        
        result = []
        for item in selection:
            if target_function(*item) == 0:
                result.append(item)
        if result:
            return result
            
        if iteration == 10000:
            return None
        iteration += 1


In [19]:
result = run_genetic_algorithm(
    POPULATION_SIZE,
    MIN_VALUE,
    MAX_VALUE,
    NUM_ITEMS_FOR_SELECTION,
    NUM_ITEMS_FOR_MUTATION,
    target_function,
    NUM_OF_SUITABLE_DESCEDANTS,
    MUTATION_PROBABILITY_FOR_SUITABLE,
    MUTATION_PROBABILITY_FOR_UNSUITABLE
)


MIN of target values: 59; AVG of target values: 7691653702000.94
MIN of target values: 0; AVG of target values: 9388124309269.967


In [20]:
result

[[67, 57, 0, 40, -256]]

#### `(u^2 * w^2 * x * y * z) + (u * w^2 * x * y * z) + (y^2 * z) + z + (x^2 * y^2 * z) = 13`

In [22]:
TARGET_VALUE = 13


def target_function(u, w, x, y, z):
    return abs(u**2*w**2*x*y*z + u*w**2*x*y*z + y**2*z + z + x**2*y**2*z - TARGET_VALUE)


In [25]:
result = run_genetic_algorithm(
    POPULATION_SIZE,
    MIN_VALUE,
    MAX_VALUE,
    NUM_ITEMS_FOR_SELECTION,
    NUM_ITEMS_FOR_MUTATION,
    target_function,
    NUM_OF_SUITABLE_DESCEDANTS,
    MUTATION_PROBABILITY_FOR_SUITABLE,
    MUTATION_PROBABILITY_FOR_UNSUITABLE
)

MIN of target values: 13; AVG of target values: 9234579553279.82
MIN of target values: 13; AVG of target values: 9207174758465.797
MIN of target values: 7; AVG of target values: 8415167200486.194
MIN of target values: 8; AVG of target values: 11092905126629.746
MIN of target values: 13; AVG of target values: 7853367551798.89
MIN of target values: 13; AVG of target values: 9787099219436.02
MIN of target values: 7; AVG of target values: 10454810720993.396
MIN of target values: 13; AVG of target values: 10461770457664.186
MIN of target values: 8; AVG of target values: 10318622196450.2
MIN of target values: 13; AVG of target values: 10282775393872.197
MIN of target values: 13; AVG of target values: 9093881897295.059
MIN of target values: 13; AVG of target values: 9615426688185.398
MIN of target values: 7; AVG of target values: 9310758454849.932
MIN of target values: 13; AVG of target values: 8162429075825.686
MIN of target values: 3; AVG of target values: 12381474055768.512
MIN of target v

In [26]:
result

[[-282, 289, 87, 0, 13]]